In [2]:
import numpy as np
from gr_pursuer.astar import astar2d
from gr_pursuer.evader import Evader
from multigrid.envs.evader_pursuer import EvaderPursuerEnv

In [3]:
env = EvaderPursuerEnv(size=16, agent_view_size=5, render_mode='human')
# env.agents[0].view_size = 5
env.reset()

evader = Evader(env.agents[1], env.goal)
observations, infos = env.reset()
# observations = [{"grid": env.grid.state, "pos": agent.pos, "dir": agent.dir} 
#                    for agent in env.agents]

: 

In [19]:
env.agents[0].pos

(8, 2)

In [ ]:
env.agents[0].pos

In [5]:
env.agents[0].view_size

7

In [21]:
a = observations[0]["image"]
print(a)

[[[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 8  1  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[10  1  1]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]

 [[ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]
  [ 1  0  0]]]


: 

In [ ]:
print(np.unique(a))

In [3]:
grid = env.grid.state[:, :, 0]
cost = (grid==2).astype(int)*1000
path = astar2d(observations[1]["pos"], env.goal, cost)
path

[[8, 8],
 [7, 8],
 [6, 8],
 [5, 8],
 [4, 8],
 [3, 8],
 [2, 8],
 [2, 9],
 [2, 10],
 [2, 11],
 [2, 12],
 [2, 13],
 [2, 14]]

In [8]:
i = path.index([1, 8])

ValueError: [1, 8] is not in list

: 

In [4]:
observations[1]["pos"]

(8, 8)

In [7]:
obs = observations[0]

In [10]:
obs

{'image': array([[[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
        [[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
        [[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
        [[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [10,  1,  1],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
        [[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
        [[ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0],
         [ 1,  0,  0]],
 
       

In [14]:
print(str(env))

WGWGWGWGWGWGWGWGWGWGWGWGWGWGWGWG
WG                            WG
WG                            WG
WG                            WG
WG                            WG
WG                            WG
WG      GG      VV            WG
WG                            WG
WG              VV            WG
WG                            WG
WG                            WG
WG                            WG
WG                          GGWG
WG            GG              WG
WG                            WG
WGWGWGWGWGWGWGWGWGWGWGWGWGWGWGWG


In [17]:
from multigrid.core.constants import IDX_TO_OBJECT
IDX_TO_OBJECT

{0: <Type.unseen: 'unseen'>,
 1: <Type.empty: 'empty'>,
 2: <Type.wall: 'wall'>,
 3: <Type.floor: 'floor'>,
 4: <Type.door: 'door'>,
 5: <Type.key: 'key'>,
 6: <Type.ball: 'ball'>,
 7: <Type.box: 'box'>,
 8: <Type.goal: 'goal'>,
 9: <Type.lava: 'lava'>,
 10: <Type.agent: 'agent'>}

In [25]:
grid = env.grid.state[:, :, 0]
grid

array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 1, 2],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]])

In [29]:
cost = (grid==2).astype(int)*1000

In [24]:
env.agents[0].dir

<Direction.down: 1>